In [4]:
import pandas as pd
import re

In [9]:
def rearrange_title(title):
    # Regex to identify titles with multiple parentheses and reorder them
    complex_pattern = re.compile(r'^(.*),\s*(The|A|An|Les)\s*\(([^)]+)\)\s*\((\d{4})\)$')
    simple_pattern = re.compile(r'^(.*),\s*(The|A|An|Les)\s*(\(\d{4}\))$')

    complex_match = complex_pattern.match(title)
    simple_match = simple_pattern.match(title)

    if complex_match:
        base_title = complex_match.group(1)
        prefix = complex_match.group(2)
        year = complex_match.group(4)
        return f"{prefix} {base_title} ({year})"
    elif simple_match:
        base_title = simple_match.group(1)
        prefix = simple_match.group(2)
        year = simple_match.group(3)
        return f"{prefix} {base_title} {year}"
    else:
        return title

def extract_year_and_clean_title(title):
    # Regular expression to find the year and any additional details in the title
    year_pattern = re.compile(r'\((\d{4})\)$')
    extra_details_pattern = re.compile(r'\s*\([^)]+\)\s*')

    # Extract the year
    year_match = year_pattern.search(title)
    if year_match:
        year = year_match.group(1)
        # Remove the year from the title
        title = year_pattern.sub('', title)
    else:
        year = None

    # Remove any additional details from the title
    title = extra_details_pattern.sub('', title).strip()

    return title, year



In [11]:
def preprocess_movie_dataset(file_path, output_path):
    # Load the dataset
    movies_df = pd.read_csv(file_path)

    # Apply the function to rearrange titles first
    movies_df['title'] = movies_df['title'].apply(rearrange_title)

    # Apply the function to extract year and clean the title
    new_data = movies_df['title'].apply(lambda x: pd.Series(extract_year_and_clean_title(x), index=['title', 'year']))

    # Assign the cleaned titles and year to the dataframe
    movies_df['title'] = new_data['title']
    movies_df['year'] = new_data['year']
    
    movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

    # Save the preprocessed dataset
    movies_df.to_csv(output_path, index=False)
    print("Dataset saved to", output_path)

# Set the path to your original dataset
input_file_path = 'ml-25m/movies.csv'
output_file_path = 'ml-25m/movies2.csv'

# Run the preprocessing function
preprocess_movie_dataset(input_file_path, output_file_path)

Dataset saved to ml-25m/movies2.csv


In [3]:
import pandas as pd

# Load the CSV files into Pandas DataFrames
links_df = pd.read_csv('ml-25m/links.csv')
movies2_df = pd.read_csv('ml-25m/movies2.csv')

# Print column names to verify
print("Links DataFrame Columns:", links_df.columns)
print("Movies2 DataFrame Columns:", movies2_df.columns)

# Check if 'imdbId' is in links_df and 'movieId' is in both
if 'imdbId' in links_df.columns and 'movieId' in links_df.columns and 'movieId' in movies2_df.columns:
    # Merge the dataframes on the 'movieId' column
    merged_df = pd.merge(movies2_df, links_df[['movieId', 'imdbId']], on='movieId', how='left')

    # Save the merged dataframe to a new CSV file
    merged_df.to_csv('movies3.csv', index=False)
    print("done")
else:
    print("Error: Necessary columns not found in dataframes.")


Links DataFrame Columns: Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')
Movies2 DataFrame Columns: Index(['movieId', 'title', 'genres', 'year'], dtype='object')
done


In [6]:
import pandas as pd

def format_imdb_id(num_id):
    # Formats the number as an IMDb ID with the 'tt' prefix and zero-padding
    formatted_id = f'tt{num_id:07d}'
    return formatted_id

# Load the CSV file into a DataFrame
movies_df = pd.read_csv('ml-25m/movies3.csv', header=0)

# Strip any whitespace from column headers
movies_df.columns = movies_df.columns.str.strip()

# Apply the formatting function to the 'imdbId' column and replace it
movies_df['imdbId'] = movies_df['imdbId'].apply(format_imdb_id)

# Save the DataFrame back to the same CSV file or to a new file if preferred
movies_df.to_csv('ml-25m/movies4.csv', index=False)  # Overwrites the original file

# Optionally print the first few rows to verify the changes
print(movies_df[['imdbId']].head())


      imdbId
0  tt0114709
1  tt0113497
2  tt0113228
3  tt0114885
4  tt0113041
